To use the examples in this chapter, first run the code below to import the PyCh Library.

In [7]:
from PyCh import *

PyCh version 0.1 imported succesfully.
 


# 8 Channels
In the previous chapter processes have been introduced. This chapter describes channels.
A channel connects two processes and is used for the transfer of data or just signals.
One process is the sending process, the other process is the receiving process.
Communication between the processes takes place instantly when both processes are willing to communicate,
 this is called *synchronous* communication.



## 8.1 A channel

We can send an entity over a channel by using `Channel.send(entity)`,  and we can receive over the same channel by using `Channel.receive()`. These are called communication events. Unlike timeout events or processes, communication events do not start as soon as they are defined. Instead, `Environment.execute(communication event)` is used to start the communication event. 

Communication over a channel requires a process which is sending, and a process which is receiving. If a process has no other process to communicate with, then it will have to wait. Communication is only completed once the receiving process has received the entity from the sending process. If a process uses the `yield` statement, such as with `yield environment.execute(communication event)`, then the process waits till communication has completed (from both sides!) before it continues.

A channel can transmit any object as entity; for example an integer, a string, or a custom data type. It is even possible to send a signal without data by using `channel.send(None)`, these signals are called *synchronization signals*, as they just synchronize actions between different processes.



### 8.1.1 Example 1

The figure below shows the two processes `P` and `C`, connected by channel `a`. Processes are denoted by circles, and channels are denoted by directed arrows in the figure. The arrow denotes the direction of communication. Process `P` is the sender or producer, process `C` is the receiver or consumer.

| Figure 1: A producer and a consumer | 
- 
<img src="figures/8-1.png" width=75%>
<a id='fig:8-1'></a>

In the model below, the producer `P` sends a stream of five integer values to consumer `C`. 
The producer sends integers `i` over channel `a`, which is realized through `env.execute(a.send(i))`. It sends the sequence of values `0, 1, 2, 3, 4` one after the other. The consumer receives these values one by one over channel `a` with `x = yield env.execute(a.receive())`. It then prints these values as output.

In [8]:
@process
def producer(env, a):
    for i in range(5):
        yield env.execute(a.send(i))

@process
def consumer(env, a):
    while True:
        i = yield env.execute(a.receive())
        print(i)

def model():
    env = Environment()
    a = Channel(env)
    P = producer(env, a)
    C = consumer(env, a)

    env.run()
    
model()

0
1
2
3
4


--- 
### 8.1.2 Example 2
Below we see an example in which synchronization signals are used; the producer does not send any data (it sends `None`). We first define our communication events through `sending = a.send(None)` and `receiving = a.receive()`, before executing and yielding them. This also allows us to later access the received entity through `receiving.entity` (which in this example is `None`).

In [9]:
@process
def producer(env, a):
    for i in range(5):
        sending = a.send(None)
        yield env.execute(sending)

@process
def consumer(env, a):
    while True:
        receiving = a.receive()
        yield env.execute(receiving)
        print(receiving.entity)

def model():
    env = Environment()
    a = Channel(env)
    P = producer(env, a)
    C = consumer(env, a)

    env.run()
    
model()

None
None
None
None
None


## 8.2 Two channels
A process can have more than one channel, allowing interaction with several other processes.

The next example shows two channels, `a` and `b`, and three processes, generator `G`, server `S` and exit `E`. Process `G` is connected via channel `a` to process `S` and process `S` is connected via channel `b` to process `E`.
The model is given in the figure below. 


| Figure 2: A generator, a server and an exit | 
- 
<img src="figures/8-2.png" width=75%>
<a id='fig:8-2'></a>

The model below shows an example of such a configuration. Process `G` sends a stream of integer values `0, 1, 2, 3, 4` to another process via channel `a`. 
Process `S` receives a value via channel `a`, assigns this value to variable `x`, doubles the value of the variable, and sends the value of the variable via `b` to another process.
Process `E` receives a value via channel `b`, assigns this value to the variable `x`, and prints this value.

After printing these five lines, process `G` stops, and processes `S` and `E` are no longer able to receive anything, so the simulation ends.


In [10]:
@process
def Generator(env, a):
    for x in range(5):
        sending = a.send(x)
        yield env.execute(sending)

@process
def Server(env, a, b):
    while True:
        receiving = a.receive()
        x = yield env.execute(receiving)
        
        sending = b.send(2*x)
        yield env.execute(sending)

@process
def Exit(env, b):
    while True:
        receiving = b.receive()
        x = yield env.execute(receiving)
        print('Exit received', x)

def model():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    
    G = Generator(env, a)
    S = Server(env, a, b)
    E = Exit(env, b)
    
    env.run()
    
model()

Exit received 0
Exit received 2
Exit received 4
Exit received 6
Exit received 8


In [11]:
def model():
    env = Environment()
    a = [Channel(env) for i in range(10)]
    b = Channel(env)
    
    G = Generator(env, a[0])
    S = Server(env, a, b)
    E = Exit(env, b)
    
    env.run()
    

## 8.3 More senders or receivers
Channels send a message (or a signal in case of synchronization channels) from one sender to one receiver.
It is however allowed to give the same channel to several sender or receiver processes. The channel selects a sender and a receiver before each communication.

The following example gives an illustration, see the figure below.

| Figure 3: A generator, two servers and an exit | 
- 
<img src="figures/8-3.png" width=75%>
<a id='fig:8-3'></a>

Suppose that only `G` and `S0` want to communicate. The channel can select a sender (namely `G`) and a receiver (process `S0`), and let both processes communicate with each other.
When sender `G`, and both receivers (`S0` and `S1`), want to communicate,
the channel selects a sender (`G` as it is the only sender available to the channel), and a receiver (either process `S0` or process `S1`), and it lets the selected processes communicate with each other. This selection process is random.

Sharing a channel in this way allows to send data to receiving processes where the receiving party is not relevant (either server process will do).
This way of communication is different from *broadcasting*, where both servers receive the same data value. Broadcasting is not supported by PyCh.

In case of two senders, `S0` and `S1`, and one receiver `E` the selection process is the same.
If one of the two servers `S` can communicate with exit `E`, communication between that server and the exit takes place.
If both servers can communicate, a random choice is made.

Having several senders and several receivers for a single channel is also handled in the same manner.
A random choice is made for the sending process and a random choice is made for the receiving process before each communication.

To communicate with several other processes but without non-determinism, unique channels must be used.

Below is the model for the configuration with two servers. We can re-use the process functions defined in the one server example; only the model needs to be redefined.

In [12]:
def model():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    
    G = Generator(env, a)
    S1 = Server(env, a, b)
    S2 = Server(env, a, b)
    E = Exit(env, b)
    
    env.run()
    
model()

Exit received 0
Exit received 2
Exit received 4
Exit received 6
Exit received 8


## 8.4 Many channels
Multiple channels can also be defined at once using list comprehension.
This can be useful in combination with list comprehension for processes.

Below is an example of using list comprehension to quickly create three channels at once for three parallel production processes.

| Figure 4: Three parallel production processes | 
- 
<img src="figures/8-4.png" width=50%>
<a id='fig:8-4'></a>

The model for this example is as follows:

In [17]:
def model():
    N = 3
    
    env = Environment()
    a = Channel(env)
    b = [Channel(env) for i in range(N)]
    c = Channel(env)
    
    G = Generator(env, a)
    S012 = [Server(env, a, b[i]) for i in range(N)]
    S345 = [Server(env, b[i], c) for i in range(N)]
    E = Exit(env, c)
    
    env.run()
    
model()

Exit received 0
Exit received 4
Exit received 8
Exit received 12
Exit received 16


## 8.5 The select statement
...


## 8.X Summary

- A channel is defined with `channel = Channel(environment)`
- Communication over a channel is done using communication events.
    - A process can send an entity over a channel with communication event `sending = channel.send(entity)`
    - A process can receive over a channel with communication event `receiving = channel.receive()` 
    - Communication events do not start automatically, they need to be executed through `env.execute(communication event)`
- A channel can transmit any type of entity; for example integers, strings, or even a custom data type
    - a Signal without data is named a *synchronization signal* and is done through: `channel.send(None)`
- A channel can have several processes sending and/or receiving.
    - If multiple processes want to send/receive at the same time, then the channel will select a sender and a receiver at random.
- Multiple channels can be defined using list comprehension, which can be useful in combination with list comprehension for processes.
	- An example would be `channels = [Channel(env) for i in range(10)]`
- A process can select one of multiple communication events. Only the first communication event which is able to communicate will be executed, the others are aborted. If multiple communication events are able to communicate at the same time, then one is chosen at random. This can be done through:
    - `env.select(communication event 1, communication event 2, ...)`
    - `env.select(*list of communication events)`
    - or with a combination of the two: `env.select(*list of communication events 1-3, communication event 4, ...)`

## 8.X Exercises
1. Given is the specification of process `P` and model `M`. Why does the model terminate immediately?



In [15]:
@process
def P(env, a, b):
    x = 0
    while True:
        x = yield env.execute(a.receive())
        x = x + 1
        print(x)   
        sending = a.send(x)
        yield env.execute(b.send())

def M():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
   
    P1 = P(a,b)
    P2 = P(b,a)

    env.run()


2. Six children have been given the assignment to perform a series of calculations on the numbers $0,1,2,3,\ldots ,9$, namely add 2, multiply by 3, multiply by 2, and add 6 subsequently. They decide to split up the calculations and to operate in parallel. They sit down at a table next to each other. The first child, the reader $R$, reads the numbers $0,1,2,3,\ldots ,9$ one by one to the first calculating child $C_1$. Child $C_1$ adds 2 and tells the result to its right neighbour, child $C_2$.  After telling the result to child $C_2$, child $C_1$ is able to start calculating on the next number the reader $R$ tells him. Children $C_2$, $C_3$, and $C_4$ are analogous to child $C_1$; they each perform a different calculation on a number they hear and tell the result to their right neighbour. At the end of the table the writer $W$ writes every result he hears down on paper. The figure below shows a schematic drawing of the children at the table.


| Figure 5: The six children |
- 
<img src="figures/Fivechild.png" width=75%>
<a id='fig:8-4'></a>

a. Finish the specification for the reading child $R$, that reads the numbers 0 till 9 one by one.

In [24]:
@process
def R(env, ...):
    i = 0
    while i < 10:
        ...

b. Specify the parameterized process $C_{\textit{add}}$ that represents the children $C_1$ and $C_4$, who perform an addition.

In [ ]:
@process
def C_add(env, ...):
    while True:
        ...

c. Specify the parameterized process $C_{\textit{mul}}$ that represents the children $C_2$ and $C_3$, who perform a multiplication.

In [ ]:
@process
def C_mul(env, ...):
    while True:
        ...

d. Specify the process $W$ representing the writing child. Print each result as output.

In [ ]:
@process
def W(env, a):
    while True:
        ...

e. Make a graphical representation of the model `SixChildren` that is composed of the six children.

f. Specify the model `SixChildren`. Simulate the model.

In [ ]:
def SixChildren():
    env = Environment()
    
    ...

    env.run()
    
SixChildren()

<details>
    <summary>[Click for the answer to a]</summary>

    Answer:
    
```python
@process
def R(env, a):
    i = 0
    while i < 10:
        yield env.execute(a.send(i))
```
</details>
    
<details>
    <summary>[Click for the answer to b]</summary>

    Answer:
    
```python
@process
def C_add(env, a, b, addition):
    while True:
        x = yield env.execute(a.receive())
        x = x + addition
        yield env.execute(b.send(x))
```
  
</details>
    
<details>
    <summary>[Click for the answer to c]</summary>

    Answer:
    
```python
@process
def C_mul(env, a, b, multiplier):
    while True:
        x = yield env.execute(a.receive())
        x = x * multiplier
        yield env.execute(b.send(x))
```
</details>
    
<details>
    <summary>[Click for the answer to d]</summary>

    Answer:
    
```python
@process
def W(env, a):
    while True:
        x = yield env.execute(a.receive())
        print(a)
```
</details>

<details>
    <summary>[Click for the answer to e]</summary>

    Answer:

    | Figure 5: The six children |
    -
    <img src="figures/8-5.png" width=75%>
    <a id='fig:8-5'></a>


</details>

<details>
    <summary>[Click for the answer to f]</summary>

    Answer:
    
```python
def SixChildren():
    env = Environment()
    
    a = Channel(env)
    b = Channel(env)
    c = Channel(env)
    d = Channel(env)
    e = Channel(env)
    
    reader = R(env, a)
    C1 = C_add(env, a, b, 2)
    C2 = C_mul(env, b, c, 3)
    C3 = C_mul(env, c, d, 2)
    C4 = C_add(env, d, e, 6)
    writer = W(env, e)
    
    env.run()
    
SixChildren()
```
</details>
    